<a href="https://colab.research.google.com/github/vsan46/gear_gacha_simulation/blob/main/Gear_through_Gacha_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import json
import csv
from collections import defaultdict

# --- CONFIGURATION ---
# Adjust these parameters to test different economic models.

# Simulation Controls
NUM_USERS = 10000
# Choose a user to see the detailed daily report for (from 0 to NUM_USERS - 1)
USER_ID_TO_PRINT = 46
TOTAL_DAYS = 200   # Total number of days to run the simulation for

# Daily Gacha Earnings
DAILY_RARE_CHESTS = 2
DAILY_EPIC_CHESTS = 1

# Gacha Probabilities
RARE_CHEST_PROBS = {
    'Common 1': 0.60,
    'Rare 1': 0.40
}
EPIC_CHEST_PROBS = {
    'Rare 1': 0.60,
    'Epic 1': 0.35,
    'Epic 2': 0.05
}

# Max Level per Rarity
MAX_LEVEL_PER_RARITY = {
    'Common 1': 5,
    'Rare 1': 15,
    'Epic 1': 30,
    'Epic 2': 50,
    'Legendary': 80,
    'Mythical': 120,
    'None': 0  # If a piece is not owned, its level contribution is 0
}

# --- END OF CONFIGURATION ---


# --- GAME DATA (Do not modify unless game design changes) ---
SETS = ['Warrior', 'Rogue', 'Defender', 'Tactician', 'Collector']
PIECES = ['Helmet', 'Chest', 'Gloves', 'Boots', 'Weapon', 'Ring']
RARITIES = ['Common 1', 'Rare 1', 'Epic 1', 'Epic 2', 'Legendary', 'Mythical']

# Generate a master list of all 30 unique items
ALL_ITEMS = [f"{s}-{p}" for s in SETS for p in PIECES]

class Gacha:
    """Handles the logic for pulling items from a gacha chest."""
    def __init__(self, probabilities):
        self.rarities = list(probabilities.keys())
        self.weights = list(probabilities.values())

    def pull(self):
        """Simulates a single pull, returning a random item and its rarity."""
        rarity = random.choices(self.rarities, self.weights, k=1)[0]
        item_name = random.choice(ALL_ITEMS)
        return item_name, rarity

class Inventory:
    """Manages a player's gear inventory and provides helper methods."""
    def __init__(self):
        self.items = {item: defaultdict(int) for item in ALL_ITEMS}

    def add_item(self, item_name, rarity, count=1):
        self.items[item_name][rarity] += count

    def remove_item(self, item_name, rarity, count=1):
        if self.items[item_name][rarity] >= count:
            self.items[item_name][rarity] -= count
            return True
        return False

    def get_top_rarity(self, item_name):
        for rarity in reversed(RARITIES):
            if self.items[item_name][rarity] > 0:
                return rarity
        return "None"

    def get_top_rarity_per_piece(self, piece_type):
        max_rarity_index = -1
        items_of_type = [f"{s}-{piece_type}" for s in SETS]

        for item_name in items_of_type:
            top_rarity_str = self.get_top_rarity(item_name)
            if top_rarity_str != "None":
                current_index = RARITIES.index(top_rarity_str)
                if current_index > max_rarity_index:
                    max_rarity_index = current_index

        if max_rarity_index == -1:
            return "None"
        else:
            return RARITIES[max_rarity_index]

class Merger:
    """Contains all the logic for merging gear pieces within an inventory."""
    def __init__(self, inventory):
        self.inventory = inventory

    def perform_all_merges(self):
        while True:
            merges_made_in_cycle = 0
            merges_made_in_cycle += self._merge_to_rare()
            merges_made_in_cycle += self._merge_to_epic1()
            merges_made_in_cycle += self._merge_to_epic2()
            merges_made_in_cycle += self._merge_to_legendary()
            merges_made_in_cycle += self._merge_to_mythical()
            if merges_made_in_cycle == 0:
                break

    def _merge_to_rare(self):
        merges = 0
        for item in ALL_ITEMS:
            while self.inventory.items[item]['Common 1'] >= 3:
                self.inventory.remove_item(item, 'Common 1', 3)
                self.inventory.add_item(item, 'Rare 1')
                merges += 1
        return merges

    def _merge_to_epic1(self):
        merges = 0
        for item in ALL_ITEMS:
            while self.inventory.items[item]['Rare 1'] >= 3:
                self.inventory.remove_item(item, 'Rare 1', 3)
                self.inventory.add_item(item, 'Epic 1')
                merges += 1
        return merges

    def _merge_to_epic2(self):
        merges = 0
        for piece in PIECES:
            items_of_type = [f"{s}-{piece}" for s in SETS]
            fodder_pool = sum(self.inventory.items[i]['Epic 1'] for i in items_of_type)

            for item in items_of_type:
                base_items = self.inventory.items[item]['Epic 1']
                available_fodder = fodder_pool - base_items
                num_merges = min(base_items, available_fodder)
                if num_merges > 0:
                    self.inventory.remove_item(item, 'Epic 1', num_merges)
                    self._remove_fodder(items_of_type, 'Epic 1', num_merges, exclude_item=item)
                    self.inventory.add_item(item, 'Epic 2', num_merges)
                    merges += num_merges
                    fodder_pool -= num_merges * 2
        return merges

    def _merge_to_legendary(self):
        merges = 0
        for piece in PIECES:
            items_of_type = [f"{s}-{piece}" for s in SETS]
            fodder_pool = sum(self.inventory.items[i]['Epic 1'] for i in items_of_type)

            for item in items_of_type:
                base_items_e2 = self.inventory.items[item]['Epic 2']
                available_fodder_e1 = fodder_pool
                num_merges = min(base_items_e2, available_fodder_e1)
                if num_merges > 0:
                    self.inventory.remove_item(item, 'Epic 2', num_merges)
                    self._remove_fodder(items_of_type, 'Epic 1', num_merges)
                    self.inventory.add_item(item, 'Legendary', num_merges)
                    merges += num_merges
                    fodder_pool -= num_merges
        return merges

    def _merge_to_mythical(self):
        merges = 0
        for item in ALL_ITEMS:
            while self.inventory.items[item]['Legendary'] >= 3:
                self.inventory.remove_item(item, 'Legendary', 3)
                self.inventory.add_item(item, 'Mythical')
                merges += 1
        return merges

    def _remove_fodder(self, item_list, rarity, count, exclude_item=None):
        removed_count = 0
        random.shuffle(item_list)
        for item_name in item_list:
            if item_name == exclude_item: continue
            while self.inventory.items[item_name][rarity] > 0 and removed_count < count:
                self.inventory.remove_item(item_name, rarity)
                removed_count += 1
        return removed_count

def main():
    """Main function to run the simulation and store results for all users."""
    print("--- Running Cup Heroes Gear Simulation ---")
    print(f"Simulating {NUM_USERS} users over {TOTAL_DAYS} days...")

    rare_gacha = Gacha(RARE_CHEST_PROBS)
    epic_gacha = Gacha(EPIC_CHEST_PROBS)

    simulation_results = []

    for user_id in range(NUM_USERS):
        inventory = Inventory()
        merger = Merger(inventory)
        user_daily_reports = []

        for day in range(1, TOTAL_DAYS + 1):
            for _ in range(DAILY_RARE_CHESTS):
                item, rarity = rare_gacha.pull()
                inventory.add_item(item, rarity)
            for _ in range(DAILY_EPIC_CHESTS):
                item, rarity = epic_gacha.pull()
                inventory.add_item(item, rarity)

            merger.perform_all_merges()

            daily_report = {}
            total_theoretical_level = 0
            piece_reports = {}
            for piece in PIECES:
                top_rarity = inventory.get_top_rarity_per_piece(piece)
                max_level = MAX_LEVEL_PER_RARITY.get(top_rarity, 0)
                total_theoretical_level += max_level
                piece_reports[piece] = top_rarity

            daily_report['day'] = day
            daily_report['pieces'] = piece_reports
            daily_report['total_level'] = total_theoretical_level
            user_daily_reports.append(daily_report)

        simulation_results.append(user_daily_reports)

        if (user_id + 1) % 1000 == 0:
            print(f"  ...simulated {user_id + 1}/{NUM_USERS} users.")

    print("\n--- Simulation Complete ---")

    # --- SAVING PHASE ---
    csv_file_name = 'simulation_results.csv'
    csv_headers = ['user_id', 'day'] + PIECES + ['total_theoretical_level']

    try:
        with open(csv_file_name, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(csv_headers)

            for user_id, user_data in enumerate(simulation_results):
                for daily_data in user_data:
                    row = [user_id, daily_data['day']]
                    for piece in PIECES:
                        row.append(daily_data['pieces'][piece])
                    row.append(daily_data['total_level'])
                    writer.writerow(row)
        print(f"\nSimulation data successfully saved to '{csv_file_name}'")
    except IOError:
        print(f"\nError: Could not write to file '{csv_file_name}'.")


    # --- REPORTING PHASE ---
    if not 0 <= USER_ID_TO_PRINT < NUM_USERS:
        print(f"\nError: USER_ID_TO_PRINT is set to {USER_ID_TO_PRINT}, which is invalid.")
        print(f"Please choose a value between 0 and {NUM_USERS - 1}.")
        return

    user_to_print_data = simulation_results[USER_ID_TO_PRINT]

    print(f"\n--- Daily Progression Report for User #{USER_ID_TO_PRINT} over {TOTAL_DAYS} days ---\n")

    for daily_report in user_to_print_data:
        day = daily_report['day']
        pieces = daily_report['pieces']
        total_level = daily_report['total_level']

        print(f"--- Day {day} ---")
        for piece, rarity in pieces.items():
            print(f"  {piece:<10}: {rarity}")

        print(f"  -----------------------")
        print(f"  Total Theoretical Level: {total_level}")
        print("-" * 25)

if __name__ == '__main__':
    main()

--- Running Cup Heroes Gear Simulation ---
Simulating 10000 users over 200 days...
  ...simulated 1000/10000 users.
  ...simulated 2000/10000 users.
  ...simulated 3000/10000 users.
  ...simulated 4000/10000 users.
  ...simulated 5000/10000 users.
  ...simulated 6000/10000 users.
  ...simulated 7000/10000 users.
  ...simulated 8000/10000 users.
  ...simulated 9000/10000 users.
  ...simulated 10000/10000 users.

--- Simulation Complete ---

Simulation data successfully saved to 'simulation_results.csv'

--- Daily Progression Report for User #46 over 200 days ---

--- Day 1 ---
  Helmet    : Rare 1
  Chest     : None
  Gloves    : None
  Boots     : None
  Weapon    : Rare 1
  Ring      : None
  -----------------------
  Total Theoretical Level: 30
-------------------------
--- Day 2 ---
  Helmet    : Rare 1
  Chest     : None
  Gloves    : None
  Boots     : None
  Weapon    : Rare 1
  Ring      : Rare 1
  -----------------------
  Total Theoretical Level: 45
-------------------------
-

In [ ]:
# Importing libraries for analysis.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Importing libraries for analysis.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Loading the .csv into memory to be able to query it
df = pd.read_csv('simulation_results.csv')

# Calculate average and median total_theoretical_level for each day
daily_level_stats = df.groupby('day')['total_theoretical_level'].agg(['mean', 'median'])

# Rename columns for clarity
daily_level_stats = daily_level_stats.rename(columns={'mean': 'Average Total Theoretical Level', 'median': 'Median Total Theoretical Level'})

# Display the results
pd.set_option('display.max_rows', None)
print("Average and Median Total Theoretical Level per Day:")
display(daily_level_stats)

Average and Median Total Theoretical Level per Day:


,Average Total Theoretical Level,Median Total Theoretical Level
day,,
1,36.0210,35.0
2,62.4225,60.0
3,82.6765,80.0
4,98.8110,95.0
5,112.7635,110.0
6,124.6445,120.0
7,135.9100,135.0
8,146.6620,145.0
9,156.7390,155.0
